In [1]:
import pandas as pd
import numpy as np
import datetime
import math

In [2]:
comments_df = pd.read_csv('../data/comments_df.csv')

In [3]:
comments_df.head(15)

,Unnamed: 0,id,parent_id,username,time_created,flair,body,tokens_new,normalized_tokens,normalized_tokens_count,word_count
0,0,14m8mf4,NaN,Tungsten_,2023-06-29 10:54:44,NaN,[Megathread] Supreme Court Ruling on Affirmati...,"['Megathread', 'Supreme', 'Court', 'Ruling', '...","['megathread', 'supreme', 'court', 'ruling', '...",62,78
1,1,jq5du0z,t3_14m8mf4,Tungsten_,2023-06-30 11:33:11,NaN,Thanks to everyone who engaged in insightful a...,"['Thanks', 'to', 'everyone', 'who', 'engaged',...","['thank', 'engage', 'insightful', 'respectful'...",9,20
2,2,jq0dgzx,t3_14m8mf4,ProudBlackMatt,2023-06-29 11:16:15,Chinese-American,I would prefer using a process that takes into...,"['I', 'would', 'prefer', 'using', 'a', 'proces...","['prefer', 'process', 'take', 'account', 'pove...",52,103
3,3,jq0cg7k,t3_14m8mf4,TomatoCanned,2023-06-29 11:09:47,NaN,"u/Tungsten_, Thanks for creating a section jus...","['u/Tungsten_,', 'Thanks', 'for', 'creating', ...","['u/tungsten_,', 'thank', 'create', 'section',...",126,269
4,4,jq0f52k,t3_14m8mf4,bad-fengshui,2023-06-29 11:26:41,NaN,As with anything related to Asians in politics...,"['As', 'with', 'anything', 'related', 'to', 'A...","['relate', 'asians', 'politic', 'm', 'see', 'l...",25,59
5,5,jq0mhrx,t3_14m8mf4,Pancake_muncher,2023-06-29 12:12:39,NaN,Yet colleges will allow alumni and doners in e...,"['Yet', 'colleges', 'will', 'allow', 'alumni',...","['college', 'allow', 'alumnus', 'doner', 'easi...",19,40
6,6,jq0mlbs,t3_14m8mf4,suberry,2023-06-29 12:13:16,NaN,I just hated Affirmative Action as a distracti...,"['I', 'just', 'hated', 'Affirmative', 'Action'...","['hate', 'affirmative', 'action', 'distraction...",78,171
7,7,jq0cvvn,t3_14m8mf4,Puzzled-Painter3301,2023-06-29 11:12:31,NaN,My own feeling is that I was never in love wit...,"['My', 'own', 'feeling', 'is', 'that', 'I', 'w...","['feeling', 'love', 'affirmative', 'action', '...",102,231
8,8,jq0jtzk,t3_14m8mf4,e9967780,2023-06-29 11:55:52,NaN,Anti Asian racism whether against East Asians ...,"['Anti', 'Asian', 'racism', 'whether', 'agains...","['anti', 'asian', 'racism', 'east', 'asians', ...",21,46
9,9,jq0kfzq,t3_14m8mf4,NaN,2023-06-29 11:59:41,NaN,Can we overturn legacy and athlete admissions ...,"['Can', 'we', 'overturn', 'legacy', 'and', 'at...","['overturn', 'legacy', 'athlete', 'admission',...",15,29


## Divide comments and posts created pre-SCOTUS and post-SCOTUS ruling

In [4]:
def str_to_datetime(str_):
    if isinstance(str_, str):
        return datetime.datetime.fromisoformat(str_)
    else:
        return None
        

comments_df['time_created_datetime'] = comments_df['time_created'].apply(lambda x: str_to_datetime(x))

#### Impute rows without dates

In [5]:
no_datetime = comments_df[pd.isna(comments_df['time_created'])]

print(no_datetime.shape)

(0, 12)


In [6]:
ruling_date = datetime.datetime(2023, 6, 29)
print(ruling_date)

2023-06-29 00:00:00


In [7]:
first_row = comments_df.iloc[0]
first_time_str = first_row['time_created']

print(type(first_time_str))

first_datetime = datetime.datetime.fromisoformat(first_time_str)
print(type(first_datetime))

print(isinstance(first_time_str, str))

<class 'str'>
<class 'datetime.datetime'>
True


In [11]:
comments_df.head(20)

,Unnamed: 0,id,parent_id,username,time_created,flair,body,tokens_new,normalized_tokens,normalized_tokens_count,word_count,time_created_datetime
0,0,14m8mf4,NaN,Tungsten_,2023-06-29 10:54:44,NaN,[Megathread] Supreme Court Ruling on Affirmati...,"['Megathread', 'Supreme', 'Court', 'Ruling', '...","['megathread', 'supreme', 'court', 'ruling', '...",62,78,2023-06-29 10:54:44
1,1,jq5du0z,t3_14m8mf4,Tungsten_,2023-06-30 11:33:11,NaN,Thanks to everyone who engaged in insightful a...,"['Thanks', 'to', 'everyone', 'who', 'engaged',...","['thank', 'engage', 'insightful', 'respectful'...",9,20,2023-06-30 11:33:11
2,2,jq0dgzx,t3_14m8mf4,ProudBlackMatt,2023-06-29 11:16:15,Chinese-American,I would prefer using a process that takes into...,"['I', 'would', 'prefer', 'using', 'a', 'proces...","['prefer', 'process', 'take', 'account', 'pove...",52,103,2023-06-29 11:16:15
3,3,jq0cg7k,t3_14m8mf4,TomatoCanned,2023-06-29 11:09:47,NaN,"u/Tungsten_, Thanks for creating a section jus...","['u/Tungsten_,', 'Thanks', 'for', 'creating', ...","['u/tungsten_,', 'thank', 'create', 'section',...",126,269,2023-06-29 11:09:47
4,4,jq0f52k,t3_14m8mf4,bad-fengshui,2023-06-29 11:26:41,NaN,As with anything related to Asians in politics...,"['As', 'with', 'anything', 'related', 'to', 'A...","['relate', 'asians', 'politic', 'm', 'see', 'l...",25,59,2023-06-29 11:26:41
5,5,jq0mhrx,t3_14m8mf4,Pancake_muncher,2023-06-29 12:12:39,NaN,Yet colleges will allow alumni and doners in e...,"['Yet', 'colleges', 'will', 'allow', 'alumni',...","['college', 'allow', 'alumnus', 'doner', 'easi...",19,40,2023-06-29 12:12:39
6,6,jq0mlbs,t3_14m8mf4,suberry,2023-06-29 12:13:16,NaN,I just hated Affirmative Action as a distracti...,"['I', 'just', 'hated', 'Affirmative', 'Action'...","['hate', 'affirmative', 'action', 'distraction...",78,171,2023-06-29 12:13:16
7,7,jq0cvvn,t3_14m8mf4,Puzzled-Painter3301,2023-06-29 11:12:31,NaN,My own feeling is that I was never in love wit...,"['My', 'own', 'feeling', 'is', 'that', 'I', 'w...","['feeling', 'love', 'affirmative', 'action', '...",102,231,2023-06-29 11:12:31
8,8,jq0jtzk,t3_14m8mf4,e9967780,2023-06-29 11:55:52,NaN,Anti Asian racism whether against East Asians ...,"['Anti', 'Asian', 'racism', 'whether', 'agains...","['anti', 'asian', 'racism', 'east', 'asians', ...",21,46,2023-06-29 11:55:52
9,9,jq0kfzq,t3_14m8mf4,NaN,2023-06-29 11:59:41,NaN,Can we overturn legacy and athlete admissions ...,"['Can', 'we', 'overturn', 'legacy', 'and', 'at...","['overturn', 'legacy', 'athlete', 'admission',...",15,29,2023-06-29 11:59:41


In [8]:
first_datetime = comments_df.iloc[0]['time_created_datetime']
print(type(first_datetime))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [9]:
pre_scotus = comments_df[comments_df['time_created_datetime'] < datetime.datetime(2023, 6, 29)]

In [10]:
pre_scotus

,Unnamed: 0,id,parent_id,username,time_created,flair,body,tokens_new,normalized_tokens,normalized_tokens_count,word_count,time_created_datetime
337,362,hqic5b,NaN,JesusSinfulHands,2020-07-13 11:22:51,NaN,A big gap in support for affirmative action ha...,"['A', 'big', 'gap', 'in', 'support', 'for', 'a...","['big', 'gap', 'support', 'affirmative', 'acti...",10,17,2020-07-13 11:22:51
338,363,fxy6xpv,t3_hqic5b,DJIisStupid,2020-07-13 12:38:51,NaN,wonder how this is split between income levels...,"['wonder', 'how', 'this', 'is', 'split', 'betw...","['wonder', 'split', 'income', 'level', 'migrat...",6,11,2020-07-13 12:38:51
339,364,fxy3l3x,t3_hqic5b,NaN,2020-07-13 12:12:24,NaN,Very interesting that the split for affirmativ...,"['Very', 'interesting', 'that', 'the', 'split'...","['interesting', 'split', 'affirmative', 'actio...",82,171,2020-07-13 12:12:24
340,365,fxxxtp3,t3_hqic5b,JesusSinfulHands,2020-07-13 11:26:06,NaN,Taken from this [story](http://aapidata.com/bl...,"['Taken', 'from', 'this', 'story](http://aapid...","['take', 'story](http://aapidata.com', 'blog',...",81,137,2020-07-13 11:26:06
341,366,fxzzm8j,t3_hqic5b,NaN,2020-07-13 22:00:26,NaN,"Frankly as a ABC currently in college, I have ...","['Frankly', 'as', 'a', 'ABC', 'currently', 'in...","['frankly', 'abc', 'currently', 'college', 're...",54,98,2020-07-13 22:00:26
...,...,...,...,...,...,...,...,...,...,...,...,...
3388,3729,epcqlfa,t1_elt4c17,arientyse,2019-05-29 14:15:14,NaN,"Please don't refer to us as ""the blacks."" It's...","['Please', 'do', ""n't"", 'refer', 'to', 'us', '...","['refer', 'black', 'okay', 'black', 'people', ...",6,20,2019-05-29 14:15:14
3389,3730,4ziaf0,NaN,unkle,2016-08-25 07:44:54,senyorito,From Self-Interest to Collective Morality: How...,"['From', 'Self', 'Interest', 'to', 'Collective...","['self', 'interest', 'collective', 'morality',...",11,20,2016-08-25 07:44:54
3390,3731,d6witvi,t3_4ziaf0,Swordbow,2016-08-25 14:47:22,NaN,The writer takes good nuggets but integrates t...,"['The', 'writer', 'takes', 'good', 'nuggets', ...","['writer', 'take', 'good', 'nugget', 'integrat...",156,354,2016-08-25 14:47:22
3391,3732,d6z9lhb,t3_4ziaf0,IRVCath,2016-08-27 17:02:04,"Fil-Am, 1.5 Generation","Interestingly enough, even though Filipinos ar...","['Interestingly', 'enough', 'even', 'though', ...","['interestingly', 'filipino', 'relatively', 'd...",51,101,2016-08-27 17:02:04


In [11]:
post_scotus = comments_df[comments_df['time_created_datetime'] >= datetime.datetime(2023, 6, 29)]

In [12]:
post_scotus

,Unnamed: 0,id,parent_id,username,time_created,flair,body,tokens_new,normalized_tokens,normalized_tokens_count,word_count,time_created_datetime
0,0,14m8mf4,NaN,Tungsten_,2023-06-29 10:54:44,NaN,[Megathread] Supreme Court Ruling on Affirmati...,"['Megathread', 'Supreme', 'Court', 'Ruling', '...","['megathread', 'supreme', 'court', 'ruling', '...",62,78,2023-06-29 10:54:44
1,1,jq5du0z,t3_14m8mf4,Tungsten_,2023-06-30 11:33:11,NaN,Thanks to everyone who engaged in insightful a...,"['Thanks', 'to', 'everyone', 'who', 'engaged',...","['thank', 'engage', 'insightful', 'respectful'...",9,20,2023-06-30 11:33:11
2,2,jq0dgzx,t3_14m8mf4,ProudBlackMatt,2023-06-29 11:16:15,Chinese-American,I would prefer using a process that takes into...,"['I', 'would', 'prefer', 'using', 'a', 'proces...","['prefer', 'process', 'take', 'account', 'pove...",52,103,2023-06-29 11:16:15
3,3,jq0cg7k,t3_14m8mf4,TomatoCanned,2023-06-29 11:09:47,NaN,"u/Tungsten_, Thanks for creating a section jus...","['u/Tungsten_,', 'Thanks', 'for', 'creating', ...","['u/tungsten_,', 'thank', 'create', 'section',...",126,269,2023-06-29 11:09:47
4,4,jq0f52k,t3_14m8mf4,bad-fengshui,2023-06-29 11:26:41,NaN,As with anything related to Asians in politics...,"['As', 'with', 'anything', 'related', 'to', 'A...","['relate', 'asians', 'politic', 'm', 'see', 'l...",25,59,2023-06-29 11:26:41
...,...,...,...,...,...,...,...,...,...,...,...,...
334,358,jq24mg4,t1_jq1itxi,Different-Rip-2787,2023-06-29 18:03:45,NaN,I fail to see how this is 'detrimental' to any...,"['I', 'fail', 'to', 'see', 'how', 'this', 'is'...","['fail', 'detrimental', 'black', 'brown', 'stu...",37,88,2023-06-29 18:03:45
335,359,jq3gway,t1_jq3f4ro,NaN,2023-06-30 00:57:44,NaN,>We have test scores to look at\n\nIf you beli...,"['>', 'We', 'have', 'test', 'scores', 'to', 'l...","['test', 'score', 'look', 'believe', 'well', '...",45,115,2023-06-30 00:57:44
336,361,jq2ectl,t1_jq25ucy,Different-Rip-2787,2023-06-29 19:16:06,NaN,> Those same options were also available to As...,"['>', 'Those', 'same', 'options', 'were', 'als...","['option', 'available', 'asam', 'student', 'di...",30,70,2023-06-29 19:16:06
3368,3708,1belws6,NaN,DudeGuyBroPalBuddie,2024-03-14 09:02:59,NaN,How White Supremacy uses Taiwan This is a long...,"['How', 'White', 'Supremacy', 'uses', 'Taiwan'...","['white', 'supremacy', 'use', 'taiwan', 'long'...",1235,2644,2024-03-14 09:02:59


(OLD) Breakdown of datetimes:
- Before SCOTUS decision: 2554 rows
- After SCOTUS decision: 304 rows

- time_created is NaN: 534 rows

Since we're missing a significant number of rows with a date, collect row's date from earlier post/comment

(UPDATED) Breakdown of datetimes:
- Before SCOTUS decision: 3054 rows
- After SCOTUS decision: 339 rows

- time_created is NaN: 0 rows

#### Look at the earliest June 29 posts/comments to see if they were truly created after SCOTUS decision was annoucned